# This is  a research notebook to try to find correlation between hyperwave and volume 

In [1]:
!pip install --upgrade pip
!pip install quandl

Requirement already up-to-date: pip in /home/nbuser/anaconda3_501/lib/python3.6/site-packages (18.0)
    100% |████████████████████████████████| 51kB 4.0MB/s ta 0:00:01
  Running setup.py bdist_wheel for inflection ... done
  Stored in directory: /home/nbuser/.cache/pip/wheels/9f/5a/d3/6fc3bf6516d2a3eb7e18f9f28b472110b59325f3f258fe9211
Successfully built inflection


In [2]:
import time
from datetime import datetime, date, time, timedelta
import json
import requests
import os.path as path

from scipy.spatial import ConvexHull

import quandl
quandl.ApiConfig.api_key = "cEofBzyzyihN3fj62kp4"

import pandas as pd
import numpy as np

import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='davzucky', api_key='aZw7LRJOSDcPJyIk2G0U')
# This is to avoid warning when dividing by zero
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [3]:
path.abspath('.')

'/home/nbuser/library'

In [4]:
# mydata =  pd.DataFrame(quandl.get("FRED/GDP",returns="numpy", collapse="weekly",qopts = { 'columns': ['ticker', 'date', 'close', 'open', 'low', 'high'] },))
# mydata['diff'] = mydata['open'] - mydata['close']


In [5]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(mydata)

In [6]:
# f = mydata
# # display(f['open'])
# f = f.set_index('date')

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(weekly_data )

## This is the place where you setup the symbol and phases

In [7]:
data_3DSystemsCorp ={
    'name':'3D SYSTEMS CORP',
    'data_source':'quandl',
    'symbol':'DDD',
}


data_Valeant = {
    'name':'Baush health Company. Ex Valeant',
    'data_source':'quandl',
    'symbol':'VRX',
}

data_Amazon = {
    'name':'Amazon',
    'data_source':'investopedia',
    'symbol':'AMZN',
}

data_Square = {
    'name':'Square',
    'data_source':'investopedia',
    'symbol':'SQ',
}

data_Netflix= {
    'name':'Netflix',
    'data_source':'investopedia',
    'symbol':'NFLX',
}

data_Apple = {
    'name':'Apple',
    'data_source':'investopedia',
    'symbol':'AAPL',
}

data_Intuit = {
    'name':'Intuit',
    'data_source':'investopedia',
    'symbol':'INTU',
}

data_Gilead_Sciences = {
    'name':'Gilead Sciences',
    'data_source':'investopedia',
    'symbol':'GILD',
}

data_Abiomed = {
    'name':'Abiomed',
    'data_source':'investopedia',
    'symbol':'ABMD',
}

data_Alphabet = {
    'name':'Alphabet',
    'data_source':'investopedia',
    'symbol':'GOOGL',
}

data_BTCUSD = {
    'name':'BTCUSD',
    'data_source':'CryptoCompare',
    'symbol':'BTC-USD',
}

data_DowJones = {
    'name':'Dow Jones',
    'data_source':'investopedia',
    'symbol':'^DJI',
}

data_Chevron = {
    'name':'Chevron',
    'data_source':'investopedia',
    'symbol':'CVX',
}


data_DowJones_1920_1933 = {
    'name':'Dow Jones 1920-1933',
    'data_source':'LocalData',
    'symbol':'DowJones_1920_1933',
}

data_DowJones = {
    'name':'Dow Jones Industrial',
    'data_source':'LocalData',
    'symbol':'DowJones_Full',
}


data_10Y_Treasury_Note = {
    'name':'10y_usd_treasury_note',
    'data_source':'LocalData',
    'symbol':'10y_usd_treasury_note',
}


data_Caterpilar = {
    'name':'Caterpilar',
    'data_source':'investopedia',
    'symbol':'CAT',
}
data_DR_Horton = {
    'name':'D.R. Horton',
    'data_source':'investopedia',
    'symbol':'DHI',
}


In [8]:
company_setup = data_DowJones  
name = company_setup ['name'] 
data_source = company_setup ['data_source'] 
symbol = company_setup ['symbol']

# constant used for other computation
root_date = datetime(1800, 1, 6)

# Helper function 
This section contain helper function that are here to load and clean the raw data

In [9]:
def add_weekid_and_price_is_closing_up(df):
#     df.loc[:,('date')] = pd.to_datetime(df.loc[:,('date')])
#     df = df.rename(columns={'Adj. Close':'close', 'Low':'low', 'Open':'open', 'High':'high',})
    df['is_price_closing_up'] = df.close > df.close.shift()
    df['weekId'] = df.apply (lambda row: get_nb_weeks (row),axis=1)
    if "volume" in list(df.columns.values):
        df = df.drop("volume", axis=1)
    return df.sort_values(by="date")

class Investopedia_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._symbol = symbol
        self._timeframe = timeframe
    
    def _clean_data(self, df):
        df.loc[:,('date')] = pd.to_datetime(df.loc[:,('Date')])
        df = df.rename(columns={'Adj. Close':'close', 'Low':'low', 'Open':'open', 'High':'high','Volume':'volume'})
        df = df.set_index('date')
        df['date'] = df.index
        return df

    def _fetch_data(self): 
        url_symbol = "https://www.investopedia.com/markets/api/partial/historical/?Symbol={}&Type=Historical+Prices&Timeframe={}&StartDate=Jan+01%2C+1900".format(self._symbol, self._timeframe)
        df_list = pd.read_html(url_symbol, header=0, parse_dates=True)
        df_price = df_list[0].dropna()
        return df_price
    
    def get_dataframe(self):
        raw_data = self._fetch_data()
        return self._clean_data(raw_data)

class Quandl_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._symbol = symbol
        self._timeframe = timeframe

    def _aggregate_ticker_weekly(self, df):    
        open = df.open.resample('W-MON').last()
        close = df.close.resample('W-FRI').last().resample('W-MON').last()
        high = df.high.resample('W-MON').max()
        low = df.low.resample('W-MON').min()
        vol = df.volume.resample('W-MON').sum() 
        
        weekly_data = pd.concat([open, close, high, low, vol], axis=1)
        weekly_data ['date'] = weekly_data .index
        return weekly_data

    def _fetch_daily_data(self):
        daily_tickers = quandl.get_table('WIKI/PRICES', \
                                         ticker = [self._symbol], \
                                         qopts = { 'columns': ['ticker', 'date', 'close', 'open', 'low', 'high', 'volume'] }, \
                                         date = { 'gte': '1900-01-01'}, \
                                         paginate=True) #, 'lte': '2016-12-31' 
        daily_tickers = daily_tickers.set_index('date')
        daily_tickers['date'] = daily_tickers.index
        return daily_tickers
    
    def get_dataframe(self):
        daily_dataframe = self._fetch_daily_data()
        if( self._timeframe == 'daily'):
            return daily_dataframe.dropna()
        
        return self._aggregate_ticker_weekly(daily_dataframe).dropna()

class CryptoCompare_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._symbol = symbol
        self._timeframe = timeframe

    def _aggregate_ticker_weekly(self, df):    
        open = df.open.resample('W-MON').last()
        close = df.close.resample('W-SUN').last().resample('W-MON').last()
        high = df.high.resample('W-MON').max()
        low = df.low.resample('W-MON').min()
        vol = df.volume.resample('W-MON').sum() 
        
        weekly_data = pd.concat([open, close, high, low, vol], axis=1)
        weekly_data ['date'] = weekly_data .index
        return weekly_data

    def _fetch_daily_data(self):
        from_symbol, to_symbol  = self._symbol.split('-')
        url = "https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym={}&allData=true&aggregate=3&e=CCCAGG".format( \
                                                                                                                        from_symbol, \
                                                                                                                        to_symbol)

        r = requests.get(url)
        array = json.dumps(r.json())
        
        data = json.loads(array)
        daily_tickers = pd.DataFrame(data["Data"])
        daily_tickers['date'] = pd.to_datetime(daily_tickers['time'],unit='s')
        daily_tickers = daily_tickers.rename( columns={"volumeto": "volume"})
        daily_tickers = daily_tickers.set_index('date')
        daily_tickers['date'] = daily_tickers.index
        return daily_tickers
    
    def get_dataframe(self):
        daily_dataframe = self._fetch_daily_data()
        if( self._timeframe == 'daily'):
            return daily_dataframe.dropna()
        
        return self._aggregate_ticker_weekly(daily_dataframe).dropna()

class LocalData_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._file_name = "{}.csv".format(symbol)
        self._timeframe = timeframe

    
    def get_dataframe(self):
        file_path = path.join(path.abspath('.'), 'data', self._file_name)
        df = pd.read_csv(file_path, header=0, parse_dates=True)
        df = df.rename(columns={column: column.lower() for column in df.columns})
        df.loc[:,('date')] = pd.to_datetime(df.loc[:,('date')])
        df = df.set_index('date')
        df['date'] = df.index
        return df
        

# https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&allData=true&aggregate=3&e=CCCAGG
    
sources = {
    "investopedia": Investopedia_Loader,
    "quandl": Quandl_Loader,
    "CryptoCompare": CryptoCompare_Loader,
    "LocalData": LocalData_Loader
}

def get_historical_data(symbol, source, timeframe='weekly'):
    source_class = sources[source](symbol, timeframe)
    dir(source_class)
    return source_class.get_dataframe()
    

In [10]:
# url = "https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&allData=true&aggregate=3&e=CCCAGG"
# df = pd.read_json(url)
# df = pd.DataFrame(df.locations.values.tolist())['Data']
# display(df)

# import json
# import pandas as pd
# import requests
# r = requests.get(url)
# # print r.json() #
# array = json.dumps(r.json())
# # print(data["Data"])
# data = json.loads(array)
# df = pd.DataFrame(data["Data"])
# df['date'] = pd.to_datetime(df['time'],unit='s')
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(df)

## Function that allow to calculate the weekid from a start date

In [11]:
def get_nb_weeks(row, base_date = root_date):
    return int((row["date"]-base_date).days/7)

def get_weekId_max_price(df):
    return df.loc[ df.loc[:,('close')].argmax()]['weekId']

def get_weekId_min_price(df):
    return df.loc[ df.loc[:,('close')].argmin()]['weekId']


In [12]:
def get_weekId_cartesian_product(df):
    df_from = df.loc[:,('weekId',"close")].rename(index=str, columns={"weekId": "weekId_from", "close": "close_from"})
    df_to = df.loc[:,('weekId',"close")].rename(index=str, columns={"weekId": "weekId_to", "close": "close_to"})
    df_cartesian = df_from.assign(foo=1).merge(df_to.assign(foo=1)).drop('foo', 1)

    return df_cartesian [(df_cartesian .weekId_from < df_cartesian .weekId_to)]    

# From here we start fetching the data
Above was only about setting up some basic function

In [13]:
# df_daily_price_raw = get_raw_historical_data(symbol, 'Daily')
df_weekly_price_raw = get_historical_data(symbol, data_source, 'Weekly')

df_weekly_price = add_weekid_and_price_is_closing_up(df_weekly_price_raw)

max_price_weekId = get_weekId_max_price(df_weekly_price)

df_weekly_price_until_max = df_weekly_price [(df_weekly_price.weekId <= max_price_weekId)]
min_price_before_max_weekId = get_weekId_min_price(df_weekly_price_until_max)
df_weekly_price_until_max = df_weekly_price_until_max [(df_weekly_price_until_max.weekId >= min_price_before_max_weekId)]
# weekId_close_from_to = get_weekId_cartesian_product(df_weekly_price_until_max)

In [14]:
print(max_price_weekId)
print(min_price_before_max_weekId)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_weekly_price_until_max)

11411
5039


,open,high,low,close,date,is_price_closing_up,weekId
date,,,,,,,
1896-08-07,22.81,22.81,20.76,20.76,1896-08-07,False,5039
1896-08-14,20.97,22.26,20.97,22.26,1896-08-14,True,5040
1896-08-21,22.50,22.68,22.13,22.13,1896-08-21,False,5041
1896-08-28,22.21,22.36,21.94,22.36,1896-08-28,True,5042
1896-09-04,23.15,25.13,23.15,25.13,1896-09-04,True,5043
1896-09-11,25.71,25.71,25.04,25.57,1896-09-11,True,5044
1896-09-18,25.25,25.25,24.13,25.21,1896-09-18,False,5045
1896-09-25,25.73,26.51,25.73,26.51,1896-09-25,True,5046
1896-10-02,26.33,26.33,25.99,25.99,1896-10-02,False,5047


In [15]:
# print(max_price_weekId)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(df_weekly_price)
# df_weekly_price.ix[:df_weekly_price_until_max]

In [16]:
def get_line_slope_and_origine(df, x1_col_name = 'x1', \
                               y1_col_name = 'y1', \
                               x2_col_name = 'x2', \
                               y2_col_name = 'y2', \
                               m_col_name = 'm', \
                               b_col_name = 'b'):
    df[m_col_name] = (df[y1_col_name] - df[y2_col_name]) / (df[x1_col_name] - df[x2_col_name])
    df[b_col_name] = df[y1_col_name] - (df[x1_col_name] * df[m_col_name])
    return df

def get_normalize_column(df, column_name = 'close'):
    column_name_normalize = '{}_normalize'.format(column_name )
    max_value = df.loc[df[column_name ].idxmax()][column_name ]
    min_value = df.loc[df[column_name ].idxmin()][column_name ]

    df[column_name_normalize] = (df.loc[:,(column_name)] - min_value)/ (max_value - min_value ) 
    return df

def get_mean_square_error(y_true, y_pred):
#     y_square_diff = np.square(y_true-y_pred)
    y_square_diff = y_true - y_pred
    return np.sum(y_square_diff) / len(y_true)

def nb_is_lower(y_true, y_pred):
    lower_item = y_true[y_true <= y_pred]
    return len(lower_item )
    
def get_y(x, m, b):
    return x * m + b

def calculate_mean_square_error(row, df):
    y_pred = get_y(df['weekId'], row['m'], row['b'])
    return get_mean_square_error(df['close'], y_pred)

def nb_cut_price_low(row, df):
    y_pred = get_y(df['weekId'], row['m'], row['b'])
    return nb_is_lower(df['low'], y_pred)

# def normalize_column(df, column_name = 'close'):
    

def get_hull_tangent(df_input, x_column_name = 'weekId', y_column_name = 'close'):
    df_input = get_normalize_column(df_input, 'close')
    df_input= get_normalize_column(df_input, 'weekId')
    hull = ConvexHull(df_input[[x_column_name,  y_column_name]].dropna())
    
    hull_results = [ [min(pair[0], pair[1]), max(pair[0], pair[1])] for pair in hull.simplices]
    data_from_to = [{"x1": df_input['weekId'].iloc[pair[0]], \
        "x1_date": root_date + timedelta(weeks = df_input['weekId'].iloc[pair[0]].item()), \
        "x1_normalize": df_input['weekId_normalize'].iloc[pair[0]], \
        "y1": df_input['close'].iloc[pair[0]], \
        "y1_normalize": df_input['close_normalize'].iloc[pair[0]], \
        "x2": df_input['weekId'].iloc[pair[1]], \
        "x2_date": root_date + timedelta(weeks = df_input['weekId'].iloc[pair[1]].item()), \
        "x2_normalize": df_input['weekId_normalize'].iloc[pair[1]], \
        "y2": df_input['close'].iloc[pair[1]], \
        "y2_normalize": df_input['close_normalize'].iloc[pair[1]]} for pair in hull_results]
    df = pd.DataFrame(data_from_to)    
    df = get_line_slope_and_origine(df)
    df = get_line_slope_and_origine(df, \
                                x1_col_name = 'x1_normalize', \
                                y1_col_name = 'y1_normalize', \
                                x2_col_name = 'x2_normalize', \
                                y2_col_name = 'y2_normalize', \
                                m_col_name = 'm_normalize', \
                                b_col_name = 'b_normalize')
    df['angle'] = np.rad2deg(np.arctan2(df['m'], 1))
    df['angle_normalize'] = np.rad2deg(np.arctan2(df['m_normalize'], 1))
    df['weeks'] = np.abs(df['x1'] - df['x2'])
    df['mean_error'] = df.apply(lambda row: calculate_mean_square_error (row, df_input),axis=1)
    df['nb_is_lower'] = df.apply(lambda row: nb_cut_price_low(row, df_input),axis=1)
    df['ratio_error_cut'] = df['mean_error'] / df['nb_is_lower']
    df['ratio_slope_y1_normalize'] = df['y1_normalize']/df['m_normalize']
    df['ratio_slope_y2_normalize'] = df['y2_normalize']/df['m_normalize']
    return df


# def calculate_line_standard(df):
#     df = get_line_slope_and_origine(df)
#     df['angle'] = np.rad2deg(np.arctan2(df['m'], 1))
#     df['days'] = np.abs(df['x1'] - df['x2'])
#     df['mean_square_error'] = df.apply(lambda row: calculate_mean_square_error (row, df),axis=1)
#     df['nb_is_lower'] = df.apply(lambda row: nb_cut_price_low(row, df),axis=1)
#     df['ratio_error_cut'] = df['mean_square_error'] / df['nb_is_lower']
#     df['ratio_slope_y1_normalize'] = df['y1_normalize']/df['m']
#     df['ratio_slope_y2_normalize'] = df['y2_normalize']/df['m']
#     return df

# def calculate_line_normalize(df):
#     df = get_line_slope_and_origine(df, \
#                                     x1_col_name = 'x1_normalize', \
#                                     y1_col_name = 'y1_normalize', \
#                                     x2_col_name = 'x2_normalize', \
#                                     y2_col_name = 'y2_normalize')
    
#     df['angle'] = np.rad2deg(np.arctan2(df['m'], 1))
#     df['days'] = np.abs(df['x1'] - df['x2'])
#     df['mean_square_error'] = df.apply(lambda row: calculate_mean_square_error (row, df),axis=1)
#     df['nb_is_lower'] = df.apply(lambda row: nb_cut_price_low(row, df),axis=1)
#     df['ratio_error_cut'] = df['mean_square_error'] / df['nb_is_lower']
#     df['ratio_slope_y1_normalize'] = df['y1_normalize']/df['m']
#     df['ratio_slope_y2_normalize'] = df['y2_normalize']/df['m']
#     return df


# def get_hull_tangent_standard(df):
#     df = get_normalize_column(df, 'close')
#     df = get_normalize_column(df, 'weekId')
#     return calculate_line_standard(get_hull_tangent(df))


# def get_hull_tangent_normalize(df):
#     df = get_normalize_column(df, 'close')
#     df = get_normalize_column(df, 'weekId')
#     return calculate_line_normalize(get_hull_tangent(df, 'weekId_normalize', 'close_normalize'))
    
    
def get_upper_path(df):
    row = df.iloc[0]
    upper_path = [row['x1'], row['x2']]

    while(row['x2'] != max_price_weekId):
        row = df.loc[df['x1'] == row['x2']].head().iloc[0]
        upper_path.append(row['x2'])

    return upper_path

def delete_path(df, path):
    df_cleaned = df.copy(True)
    
    for x1, x2 in zip(path[:-1], path[1:]):
        df_cleaned = df_cleaned[(np.logical_not((df_cleaned.x1 == x1) & (df_cleaned.x2 == x2))) ]

    return df_cleaned.sort_values(['x1','y2'], ascending=[True, False])

def delete_upper_path(df):
    return delete_path(df, get_upper_path(df.sort_values(['x1','x2'], ascending=[True, False])))

def delete_shorter_path(df):
    df = df.sort_values(['x1','x2'], ascending=[True, False])
    path1 = get_upper_path(df)
    path2 = get_upper_path(df.iloc[1:])
    path = path1 if len(path1) < len(path2) else path2
    return delete_path(df, path)
    

In [17]:
# hull_results = [ [min(pair[0], pair[1]), max(pair[0], pair[1])] for pair in hull.simplices]
# data_from_to = [{"x1": df_weekly_price_until_max['weekId'].iloc[pair[0]], \
#    "y1": df_weekly_price_until_max['close'].iloc[pair[0]], \
#    "x2": df_weekly_price_until_max['weekId'].iloc[pair[1]], \
#    "y2": df_weekly_price_until_max['close'].iloc[pair[1]]} for pair in hull_results]
# df = pd.DataFrame(data_from_to)

In [18]:
df_hull_tangent = get_hull_tangent(df_weekly_price_until_max)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_hull_tangent.sort_values(['x1','x2'], ascending=[True, False]))
py.iplot(graph_hyperwave(df_weekly_price_until_max,df_hull_tangent), filename='hyperwave')
# df_hull_tangent = delete_upper_path(df_hull_tangent)

,x1,x1_date,x1_normalize,x2,x2_date,x2_normalize,y1,y1_normalize,y2,y2_normalize,m,b,m_normalize,b_normalize,angle,angle_normalize,weeks,mean_error,nb_is_lower,ratio_error_cut,ratio_slope_y1_normalize,ratio_slope_y2_normalize
0,5039,1896-08-03,0.000000,11377,2018-01-22,0.994664,20.76,0.000000,26616.71,0.995255,4.196269,-2.112424e+04,1.000594,0.000000,76.596023,45.017022,6338,-1.061090e+04,6352,-1.670482,0.000000,0.994664
6,5039,1896-08-03,0.000000,6913,1932-07-04,0.294099,20.76,0.000000,41.60,0.000780,0.011121,-3.527669e+01,0.002652,0.000000,0.637137,0.151930,1874,2.751834e+03,2,1375.917034,0.000000,0.294099
7,6913,1932-07-04,0.294099,7424,1942-04-20,0.374294,41.60,0.000780,94.30,0.002752,0.103131,-6.713454e+02,0.024591,-0.006452,5.888161,1.408704,511,2.630482e+03,4,657.620482,0.031713,0.111907
9,7424,1942-04-20,0.374294,7797,1949-06-13,0.432831,94.30,0.002752,163.80,0.005353,0.186327,-1.288992e+03,0.044429,-0.013878,10.554720,2.543946,373,2.563268e+03,4,640.817098,0.061940,0.120477
8,7797,1949-06-13,0.432831,9126,1974-12-02,0.641400,163.80,0.005353,577.60,0.020838,0.311362,-2.263889e+03,0.074244,-0.026782,17.294600,4.246067,1329,2.508891e+03,5,501.778291,0.072097,0.280666
2,9126,1974-12-02,0.641400,9526,1982-08-02,0.704175,577.60,0.020838,784.30,0.028573,0.516750,-4.138260e+03,0.123218,-0.058195,27.327660,7.024482,400,2.692530e+03,4,673.132530,0.169112,0.231887
11,9526,1982-08-02,0.704175,9648,1984-12-03,0.723321,784.30,0.028573,1163.20,0.042752,3.105738,-2.880096e+04,0.740559,-0.492910,72.152236,36.522124,122,6.042954e+03,14,431.639576,0.038583,0.057729
12,9648,1984-12-03,0.723321,9689,1985-09-16,0.729755,1163.20,0.042752,1297.90,0.047792,3.285366,-3.053401e+04,0.783391,-0.523891,73.070794,38.074826,41,6.297327e+03,9,699.702947,0.054573,0.061007
10,9689,1985-09-16,0.729755,9804,1987-11-30,0.747803,1297.90,0.047792,1766.70,0.065335,4.076522,-3.819952e+04,0.972041,-0.661560,76.217098,44.187730,115,7.450124e+03,7,1064.303477,0.049167,0.067215
4,9804,1987-11-30,0.747803,9953,1990-10-08,0.771186,1766.70,0.065335,2398.00,0.088959,4.236913,-3.977199e+04,1.010286,-0.690159,76.720029,45.293160,149,7.702276e+03,8,962.784497,0.064670,0.088054


NameError: name 'graph_hyperwave' is not defined

In [ ]:
df_hull_tangent.sort_values(['x1','x2'], ascending=[True, False])

In [54]:
def graph_hyperwave(df_data, df_hyperwave):

    x = df_data['weekId'].values
    y =  df_data['close_normalize'].values
    trace0 = go.Scatter(
        x = x,
        y = y,
        mode = 'lines',
        name = 'lines'
    )

    layout = {
    #     'xaxis': {
    #         'range': [0, 7]
    #     },
    #     'yaxis': {
    #         'range': [0, 2.5]
    #     },
        'shapes': [{
                'type': 'line',
                'x0': row[1]['x1'],
                'y0': row[1]['y1'],
                'x1': row[1]['x2'],
                'y1': row[1]['y2'],
                'line': {
                    'color': 'rgb(55, 128, 191)',
                    'width': 3,
                }
            } for row in df_hyperwave.iterrows()
            # Line Vertical
            ]
    }
    data = [trace0]

    # py.iplot(data, filename='line-mode')
    fig = {
        'data': data,
        'layout': layout,
    }

    return fig
#     py.iplot(fig, filename='shapes-lines')

In [124]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(df_weekly_price_until_max)
# df =delete_upper_path(df_weekly_price_until_max)
# display(df)

class hyperwave:
    def __init__(self, \
                 min_m = 0.8, \
                 start_weeks_lookup_max = 156, \
                 phase_min_weeks = 15, \
                 slope_increase = 2):
        self.start_weeks_lookup_max = start_weeks_lookup_max
        self.slope_increase = slope_increase
        self.min_m  = min_m  
        self.phase_min_weeks = phase_min_weeks 

    def get_hyperwave(self, df):
        # Step 1 - Get the raw Hull from max and min raw data
        df_min_to_max = self._borne_raw_data_between_max_to_min( df )
        df_hull = self._delete_above_path( get_hull_tangent( df_min_to_max ))
        
        df_hw_m_greater = self._order_and_reset_index( df_hull[(df_hull.m_normalize >= self.min_m )] )
                               
        # Step 2 - Find max Price prior of start hyperwave
        min_week = df_hw_m_greater.iloc[0].x1
        max_price_weeks_before_start_week = self._get_max_price_week_before(df, min_week)
        
        hw_start_weekId = self._get_weekId_first_price_greater_than( df_min_to_max, \
                                                                    min_week, \
                                                                    max_price_weeks_before_start_week )
        
        # Step 3 - Get new Hull for the borned hyperwave raw data
        df_hyperwave_raw_data = df_min_to_max[(df_min_to_max.weekId >= hw_start_weekId )]

        df_hull_hyperwave = self._order_and_reset_index( \
                                    self._delete_above_path( get_hull_tangent( df_hyperwave_raw_data ))) 

        filtered_hw = df_hull_hyperwave[ (df_hull_hyperwave.m_normalize > 0) ]
        
        current_phase_m = filtered_hw.iloc[0].m_normalize
        hw_phases_temp = []
        hw_current_phase = [filtered_hw.index[0]]
        
        for index, row in filtered_hw.loc[2:].iterrows():
            if row.m_normalize < current_phase_m * self.slope_increase:
                hw_current_phase.append(index)
            else:
                hw_phases_temp.append(hw_current_phase)
                hw_current_phase = [index]
                current_phase_m = row.m_normalize
        
        hw_phases_temp.append(hw_current_phase)
        
        for i in np.arange(len(hw_phases_temp)-1, 1, -1):
            phase = hw_phases_temp[i]
            if self._sum_group_weeks( filtered_hw, phase) < self.phase_min_weeks:
                hw_phases_temp.remove( phase )
                hw_phases_temp[ i - 1 ].extend( phase )
        
#         display(self._sum_group_weeks( filtered_hw, hw_phases_temp[2]))
        
        return hw_phases_temp
 

    def _order_and_reset_index(self, df):
        return df.sort_values(['x1','x2'], ascending=[True, False]) \
            .reset_index()


    def _sum_group_weeks(self, df, group):
        return df.loc[group].sum()['weeks']
    
    def _get_weekId_first_price_greater_than(self, df, min_week_id, max_price):
        df_week_greater_than = df[(df.weekId >= min_week_id)]

        df_val_price_greater_than_max = df_week_greater_than[(df_week_greater_than.close > max_price)]
        return df_val_price_greater_than_max.loc[ df_val_price_greater_than_max.loc[:,('weekId')].argmin()]['weekId']

    def _delete_above_path(self, df):
# As we are using Hull to find the external phase of the graph. The positive mean_error as the way up
# whereas the negative are the way down
        return df[(df.mean_error >= 0)]

    def _borne_raw_data_between_max_to_min(self, df):
#     Born the dataframe from with all the value before weekId of Max and from them find the min to born the other side
        max_price_weekId = self._get_weekId_max_price(df)
        df_until_max = df[(df.weekId <= max_price_weekId)]
        min_price_before_max = self._get_weekId_min_price( df_until_max )    
        df_min_to_max = df_until_max  [(df_until_max.weekId >= min_price_before_max)]
        return df_min_to_max

    def _get_weekId_max_price(self, df):
        return df.loc[ df.loc[:,('close')].argmax()]['weekId']

    def _get_weekId_min_price(self, df):
        return df.loc[ df.loc[:,('close')].argmin()]['weekId']

    def _get_max_price(self, df, column_name = 'close'):
        return df.loc[ df.loc[:,(column_name)].argmax()][column_name]

    def _get_max_price_week_before(self, df, weekId):
        last_n_weeks_Items = df[(df.weekId <= weekId)].tail(self.start_weeks_lookup_max)    
        max_price = self._get_max_price( last_n_weeks_Items )
        return max_price


In [125]:
hw = hyperwave()
df_result = hw.get_hyperwave(df_weekly_price)
# display(df_result)
display(df_result)
# display(df_result.loc[2:])
# hw._get_hw_start_week(df_result)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



[[1, 2, 3], [4, 5], [6, 7, 8, 9]]

In [37]:
df = df_weekly_price_until_max
max_weekId = 9689
nb_tail_items = 156

def get_max_price(df, column_name = 'close'):
    return df.loc[ df.loc[:,(column_name)].argmax()][column_name]
# weekId_max_price = get_weekId_max_price(last_n_weeks_Items) 
#     max_price_last_n_weeks = last_n_weeks_Items[(last_n_weeks_Items.weekId == weekId_max_price )].iloc[0]

last_n_weeks_Items = df[(df.weekId <= max_weekId)].tail(nb_tail_items)    
max_price = get_max_price(last_n_weeks_Items )

print( max_price)

df_week_greater_than = df[(df.weekId >= max_weekId)]

# df.loc[ df.loc[:,('close')].argmax()]['weekId']
df_val_price_greater_than_max = df_week_greater_than[(df_week_greater_than.close > max_price)]
weekId_start_hyperwave = df_val_price_greater_than_max.loc[ df_val_price_greater_than_max.loc[:,('weekId')].argmin()]['weekId']
df_week_greater_than = df[(df.weekId >= weekId_start_hyperwave )]

df = get_normalize_column(df_week_greater_than, 'close')
df = get_normalize_column(df, 'weekId')

df_hyperwave = get_hull_tangent(df,"weekId_normalize", "close_normalize").sort_values(['x1','y2'], ascending=[True, False])


df_hyperwave = delete_shorter_path(df_hyperwave)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_hyperwave)
py.iplot(graph_hyperwave(df_weekly_price_until_max,df_hyperwave), filename='hyperwave')
# py.iplot(fig, filename='shapes-lines')

1359.5


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,x1,x1_date,x1_normalize,x2,x2_date,x2_normalize,y1,y1_normalize,y2,y2_normalize,m,b,m_normalize,b_normalize,angle,angle_normalize,weeks,mean_error,nb_is_lower,ratio_error_cut,ratio_slope_y1_normalize,ratio_slope_y2_normalize
0,9693,1985-10-14,0.000000,9694,1985-10-21,0.000582,1368.80,0.000484,1356.50,0.000000,-12.300000,1.205927e+05,-0.832371,0.000484,-85.352029,-39.773012,1,18595.006789,2,9297.503394,-0.000582,-0.000000
5,9694,1985-10-21,0.000582,9804,1987-11-30,0.064610,1356.50,0.000000,1766.70,0.016158,3.729091,-3.479331e+04,0.252357,-0.000147,74.988631,14.163256,110,4842.046789,6,807.007798,0.000000,0.064028
6,9804,1987-11-30,0.064610,9953,1990-10-08,0.151339,1766.70,0.016158,2398.00,0.041025,4.236913,-3.977199e+04,0.286722,-0.002367,76.720029,15.998772,149,4462.196051,8,557.774506,0.056354,0.143083
3,9953,1990-10-08,0.151339,10913,2009-03-02,0.710128,2398.00,0.041025,6626.94,0.207604,4.405146,-4.144642e+04,0.298107,-0.004090,77.210199,16.599683,960,4361.424435,9,484.602715,0.137618,0.696407
2,10913,2009-03-02,0.710128,11275,2016-02-08,0.920838,6626.94,0.207604,15973.84,0.575781,25.820166,-2.751485e+05,1.747313,-1.033212,87.782076,60.217184,362,12092.246623,6,2015.374437,0.118813,0.329523
10,11275,2016-02-08,0.920838,11313,2016-10-31,0.942957,15973.84,0.575781,17888.28,0.651191,50.380000,-5.520607e+05,3.409337,-2.563667,88.862877,73.652946,38,29849.006789,5,5969.801358,0.168883,0.191002
11,11313,2016-10-31,0.942957,11399,2018-06-25,0.993015,17888.28,0.651191,24271.41,0.902624,74.222442,-8.217902e+05,5.022813,-4.085106,89.228100,78.740121,86,47993.105045,5,9598.621009,0.129647,0.179705
9,11399,2018-06-25,0.993015,11409,2018-09-03,0.998836,24271.41,0.902624,25916.54,0.967426,164.513000,-1.851012e+06,11.132995,-10.152608,89.651729,84.867290,10,124469.207789,8,15558.650974,0.081076,0.086897
13,11409,2018-09-03,0.998836,11410,2018-09-10,0.999418,25916.54,0.967426,26154.67,0.976806,238.130000,-2.690909e+06,16.114836,-15.128650,89.759394,86.449086,1,187558.976789,4,46889.744197,0.060033,0.060615
12,11410,2018-09-10,0.999418,11411,2018-09-17,1.000000,26154.67,0.976806,26743.50,1.000000,588.830000,-6.692396e+06,39.847557,-38.847557,89.902696,88.562427,1,488459.576789,2,244229.788394,0.024514,0.025096


In [ ]:

# display(df_hyperwave.sort_values(['x1','y2'], ascending=[True, False]))
# print(max_price_weekId)
# print(get_upper_path(df_hyperwave))
df_low_path = delete_upper_path(df_hull_tangent)
display(df_hull_tangent)
display(df_low_path )
print(df_low_path.std()) 
print(df_low_path.mean()) #(axis=['m'])
print(df_low_path.median())

In [ ]:
# def get_line_slope(row):
#     return (row.close_from - row.close_to) / (row.weekId_from - row.weekId_to)


# weekId_close_from_to['slope'] = weekId_close_from_to.apply (lambda row: get_line_slope(row),axis=1)



# weekId_close_from_to

# # m = -0.110000
# # b = 1092.930000
# # m = 15.82
# # b = -177675.62
# def is_phase_below(m, b):
# #     m = row.m
# #     b = row.b
#     df = df_weekly_price
#     df_weekly_close = df[["weekId", "close"]]
#     df_weekly_close['ln_y'] = (df_weekly_close.weekId * m) + b
#     df_weekly_close['is_below'] = df_weekly_close.ln_y.le(df_weekly_close.close) | np.isclose(df_weekly_close['ln_y'], df_weekly_close['close'])
#     return df_weekly_close['is_below'].all()


# weekId_close_from_to['is_below'] = weekId_close_from_to.apply (lambda row: is_phase_below(row, df_weekly_price),axis=1)
# is_phase_below()

# weekId_close_from_to['r'] = weekId_close_from_to(weekId_close_from_to['m'], weekId_close_from_to['b'] )

# weekId_close_from_to
# import numpy as np
# import pandas as pd
# np.seterr(divide='ignore', invalid='ignore')



# df = df_weekly_price_until_max[:15].rename(columns={'weekId':'x', 'close': 'y'})


# data = {'x':[1,2,3,4],'y':[5,6,10,12]}
# df = pd.DataFrame(data)



# calculate m and b from y = mx = b
# df_m = (df['y'].values - df['y'].values[:, None]) / (df['x'].values - df['x'].values[:, None])
# df_b = df['y'].values - (df['x'].values * df_m)


# import itertools
# nb_partition = int((df['x'].values.shape[0] / 10)) + 1
# x_values_split = np.array_split(df['x'].values, nb_partition) 
# y_values_split = np.array_split(df['y'].values, nb_partition)              


# def get_y_values(x_value, m, b):
#     x_lenght = x_values.shape[0]
#     cube_shape = (x_lenght, 1, 1)
#     x_cube = np.reshape(x_values, cube_shape)
    
#     return (x_cube * m) + b 

# def get_is_line_above_close(x_values, y_values, m, b):
#     y_lenght = y_values.shape[0]
#     cube_shape = (y_lenght, 1, 1)
    
#     y_cube = np.reshape(y_values, cube_shape)
#     return get_y_values(x_values,m, b) < y_cube

    
# result = [get_y_vlowalues(x_values, df_m, df_b) for (x_values, y_values) in itertools.zip_longest(x_values_split, y_values_split )]
#     x_lenght = x_values.shape[0]
#     cube_shape = (x_lenght, 1, 1)
    
#     x_cube = np.reshape(x_values, cube_shape)
#     y_cube = np.reshape(y_values, cube_shape)
#     cube_y_value_calculated =  (x_cube * df_m) + df_b 

# x_values = x_values_split[0]
# y_values = y_values_split[0]              
# x_lenght = x_values.shape[0]
# cube_shape = (x_lenght, 1, 1)

# x_cube = np.reshape(x_values, cube_shape)
# y_cube = np.reshape(y_values, cube_shape)
